In [1]:
"""
testing.ipynb

File for performing testing to implement lottery ticket experiments.

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""

%load_ext tensorboard
import numpy as np
import os

from src.harness.dataset import download_data, load_and_process_mnist
from src.harness.model import create_model
from src.lottery_ticket.foundations.trainer import train

In [2]:
download_data()

In [4]:
X_train, Y_train, X_test, Y_test = load_and_process_mnist()
model = create_model(0, X_train, Y_train)

Directory 'models/model_0/' created successfully.
